In [1]:
from keras.applications import ResNet50
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

def Resnet50_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10,
                                                                    stratify=y_train)
    print(len(X_train))


    model = ResNet50(include_top=True, weights=None, input_shape=(227,227,3), pooling=max, classes=2)
    model.compile(loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'],
                  optimizer=keras.optimizers.SGD(learning_rate=0.01))
    modelhistory = model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_validation, y_validation))
    evaluation = []
    evaluation.append(model.evaluate(X_train, y_train, batch_size=16))
    evaluation.append(model.evaluate(X_test, y_test, batch_size=16))
    
    return evaluation

In [2]:
image_size = 227

groups_folder_path = '../Data/AAFT_3/'

X_s = []

y = []
# auto_corr       hadamard        midAng
for label in ['Bi', 'Tri']:

    for feature in ['striping_bw']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            # print(f)
            for filename in f:
                # print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])

                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(0)
                else:
                    y.append(1)

print(X_s[0].shape)
# print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

res = []
for i in range(10):
    res.append(Resnet50_model(X, y))

(227, 227, 3)
360
259
Epoch 1/10
17/17 [==============================] - 32s 1s/step - loss: 2.9902 - accuracy: 0.5903 - val_loss: 1.9040 - val_accuracy: 0.4828
Epoch 2/10
17/17 [==============================] - 2s 133ms/step - loss: 0.0495 - accuracy: 0.9769 - val_loss: 0.8169 - val_accuracy: 0.5172
Epoch 3/10
17/17 [==============================] - 2s 135ms/step - loss: 0.0758 - accuracy: 0.9599 - val_loss: 1.3260 - val_accuracy: 0.5172
Epoch 4/10
17/17 [==============================] - 2s 134ms/step - loss: 0.0267 - accuracy: 0.9866 - val_loss: 1.6775 - val_accuracy: 0.5172
Epoch 5/10
17/17 [==============================] - 2s 134ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 2.1662 - val_accuracy: 0.5172
Epoch 6/10
17/17 [==============================] - 2s 131ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 2.7625 - val_accuracy: 0.5172
Epoch 7/10
17/17 [==============================] - 2s 130ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 2.9055 - val_acc

17/17 [==============================] - 2s 127ms/step - loss: 0.1513 - accuracy: 0.9487 - val_loss: 1.0220 - val_accuracy: 0.5172
Epoch 3/10
17/17 [==============================] - 2s 127ms/step - loss: 0.0176 - accuracy: 0.9933 - val_loss: 1.1495 - val_accuracy: 0.5172
Epoch 4/10
17/17 [==============================] - 2s 127ms/step - loss: 0.0291 - accuracy: 0.9889 - val_loss: 4.8441 - val_accuracy: 0.5172
Epoch 5/10
17/17 [==============================] - 2s 128ms/step - loss: 3.2177 - accuracy: 0.8321 - val_loss: 2.2403 - val_accuracy: 0.5172
Epoch 6/10
17/17 [==============================] - 2s 128ms/step - loss: 0.0360 - accuracy: 0.9871 - val_loss: 1.1030 - val_accuracy: 0.5172
Epoch 7/10
17/17 [==============================] - 2s 128ms/step - loss: 0.5279 - accuracy: 0.9010 - val_loss: 1.9747 - val_accuracy: 0.5172
Epoch 8/10
17/17 [==============================] - 2s 128ms/step - loss: 0.1087 - accuracy: 0.9831 - val_loss: 4.0366 - val_accuracy: 0.5172
Epoch 9/10
17/17 

In [3]:
for data in res:
    for d in data:
        for j in d:
            print(j)

5.760599613189697
0.49806949496269226
5.73896598815918
0.5
11.08315658569336
0.49806949496269226
11.038687705993652
0.5
4.662301540374756
0.49806949496269226
4.636003017425537
0.5
5.810398578643799
0.49806949496269226
5.790943622589111
0.5
10.593381881713867
0.49806949496269226
10.553255081176758
0.5
7.376219749450684
0.49806949496269226
7.3544769287109375
0.5
5.026795387268066
0.49806949496269226
5.0096540451049805
0.5
4.469061851501465
0.49806949496269226
4.44808292388916
0.5
7.0333638191223145
0.49806949496269226
7.007992267608643
0.5
4.123653411865234
0.49806949496269226
4.109650611877441
0.5
